In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('/content/drive/MyDrive/GitHub/makemore_names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
C = torch.randn((27, 2))

In [8]:
C[5]

tensor([-0.3856,  0.3188])

In [9]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.3856,  0.3188])

In [10]:
C[X].shape

torch.Size([32, 3, 2])

In [11]:
X[13, 2]

tensor(1)

In [12]:
C[X][13, 2]

tensor([-0.5281, -0.0302])

In [13]:
C[1]

tensor([-0.5281, -0.0302])

In [14]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [15]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [16]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [17]:
h

tensor([[-0.9818, -0.9324, -0.9977,  ..., -0.9981, -0.4005,  0.8043],
        [-0.8527, -0.9849, -0.9948,  ..., -0.9794,  0.9103,  0.1435],
        [-0.7770, -0.8534, -0.8131,  ..., -0.9918, -0.9686, -0.9286],
        ...,
        [-0.7488, -0.9892, -0.9991,  ..., -0.9836,  0.9954,  0.7034],
        [-0.7115, -0.8177, -0.2245,  ..., -0.9980, -0.9586, -0.1503],
        [-0.8284,  0.1236, -0.9559,  ..., -0.8555,  0.7806,  0.8673]])

In [18]:
h.shape

torch.Size([32, 100])

In [19]:
(emb.view(-1, 6) @ W1).shape

torch.Size([32, 100])

In [20]:
b1.shape

torch.Size([100])

In [21]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [22]:
logits = h @ W2 + b2

In [23]:
logits.shape

torch.Size([32, 27])

In [24]:
counts = logits.exp()

In [25]:
prob = counts / counts.sum(1, keepdims=True)

In [26]:
prob.shape

torch.Size([32, 27])

In [27]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(15.7379)

In [28]:
# cleanup

In [29]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [30]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [31]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [48]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdims=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y)
loss

tensor(17.7697)